Load model and embedding data.

In [1]:
import theano
import theano.tensor as T
import numpy as np
import cPickle
import random
from utils import *
from state import *
from attention_model2 import AttentionModel2

theano.config.floatX='float32'

model_name = 'model/attention2_emb256_h256_v2_model.npz'

state = prototype_state()
model = AttentionModel2(state, test_mode=True)
model.load(model_name)

(word2ind, ind2word) = cPickle.load(open('tmp/dict.pkl'))

print('Data loaded.')

Data loaded.


Now we try to mannually provide an input sentence (if a word is out of vocab, we simply remove it).

In [7]:
test_sents = ['moderately priced swedish food', 'how about asian oriental', 'chinese food', \
              'moderately australian food', 'is there any serving vietnamese', \
             'can i have some catalan dish']

for (k, test_sent) in enumerate(test_sents):
    print "Test sent:", test_sent
    words = test_sent.split()
    nat_coded = [1]
    for w in words:
        if w in word2ind:
            nat_coded.append(word2ind[w])
        else:
            print '  out of vocab: %s' % w
    nat_coded.append(0)
    #print 'Coded input:', nat_coded

    m = state['seq_len_in']
    nat_coded_mat = numpy.zeros((m, 2), dtype='int32')
    nat_mask = numpy.zeros((m, 2), dtype='float32')
    sent_len = len(nat_coded)
    nat_coded_mat[:sent_len, 0] = nat_coded
    nat_mask[:sent_len, 0] = 1
    nat_coded_mat[:sent_len, 1] = nat_coded
    nat_mask[:sent_len, 1] = 1
    pred_fn = model.build_gen_function()
    
    res = [1]
    abs_in = 1
    model.gen_reset()
    while True:
        abs_in_mat = np.zeros((2, ), dtype='int32') + abs_in
        #print 'abs_in', abs_in_mat
        [p_t, o_t, alpha_t] = pred_fn(nat_coded_mat, nat_mask, abs_in_mat)
        #print "ot", o_t, ind2word[o_t[0]]
        pt_col = p_t[0]
        alpha_t = alpha_t[:, 0]
        #print alpha_t
        alpha_s = alpha_t.argsort()[::-1]
        #print sum(pt_col)
        pt_norm = [1.0 * a / sum(pt_col) for a in pt_col]
        #print pt_norm
        ind = np.asarray(pt_norm).argmax()
        abs_in = ind
        res.append(ind)
        if ind == 0 or len(res) > 10:        
            break
        print 'Explanation of: %s' % ind2word[ind]
        for k in alpha_s[:len(nat_coded)]:
            print "    %s: %.4f" % (ind2word[nat_coded[k]], alpha_t[k])
    print
    print ' '.join([ind2word[ind] for ind in res])
    print

Test sent: moderately priced swedish food
Explanation of: swedish
    swedish: 0.9031
    <END>: 0.0884
    food: 0.0062
    priced: 0.0013
    moderately: 0.0011
    <START>: 0.0000

<START> swedish <END>

Test sent: how about asian oriental
Explanation of: asian
    <END>: 0.6675
    oriental: 0.3281
    about: 0.0027
    how: 0.0017
    asian: 0.0001
    <START>: 0.0000
Explanation of: oriental
    <END>: 0.6675
    oriental: 0.3281
    about: 0.0027
    how: 0.0017
    asian: 0.0001
    <START>: 0.0000

<START> asian oriental <END>

Test sent: chinese food
Explanation of: chinese
    chinese: 0.7102
    <END>: 0.1816
    food: 0.1083
    <START>: 0.0000

<START> chinese <END>

Test sent: moderately australian food
Explanation of: australian
    australian: 0.9988
    moderately: 0.0005
    <END>: 0.0005
    food: 0.0002
    <START>: 0.0000

<START> australian <END>

Test sent: is there any serving vietnamese
Explanation of: vietnamese
    vietnamese: 0.9343
    there: 0.0618
    is